In [2]:
import torch

# 1. 随机序列

## 1.1 设置种子

<div style="background: #E0F2F1; border-left: 5px solid #2E8B57; padding: 10px; margin: 10px 0;">
<strong>TLDR</strong><br>
第一个(Generator)创建的是独立的生成器对象，可以单独控制随机流<br>
后两个是全局设置函数，影响默认的随机行为
</div>


### 1.1.1 Generator

In [2]:
# 创建固定种子的生成器
generator = torch.Generator()
generator.manual_seed(42)  # 设置种子为 42

# 使用这个生成器
w = torch.empty(3, 5)
print(torch.nn.init.normal_(w, generator=generator))

# 直接再次使用相同的生成器不会产生相同的随机数
w2 = torch.empty(3, 5)
print(torch.nn.init.normal_(w2, generator=generator))

print(torch.allclose(w, w2))  # 输出：False（生成器状态已改变）

tensor([[ 0.3367,  0.1288,  0.2345,  0.2303, -1.1229],
        [-0.1863,  2.2082, -0.6380,  0.4617,  0.2674],
        [ 0.5349,  0.8094,  1.1103, -1.6898, -0.9890]])
tensor([[ 0.9580,  1.3221,  0.8172, -0.7658, -0.7506],
        [ 1.3525,  0.6863, -0.3278,  0.7950,  0.2815],
        [ 0.0562,  0.5227, -0.2384, -0.0499,  0.5263]])
False


In [3]:
#-----------#
# 创建固定种子的生成器
generator = torch.Generator()
generator.manual_seed(42)  # 设置种子为 42
# 使用这个生成器
w = torch.empty(3, 5)
print(torch.nn.init.normal_(w, generator=generator))

# 再次设置相同的生成器
generator = torch.Generator()
generator.manual_seed(42)  # 设置种子为 42
#再次使用,产生相同的随机数
w2 = torch.empty(3, 5)
print(torch.nn.init.normal_(w2, generator=generator))

print(torch.allclose(w, w2))  # 输出：True（生成器状态未改变）

tensor([[ 0.3367,  0.1288,  0.2345,  0.2303, -1.1229],
        [-0.1863,  2.2082, -0.6380,  0.4617,  0.2674],
        [ 0.5349,  0.8094,  1.1103, -1.6898, -0.9890]])
tensor([[ 0.3367,  0.1288,  0.2345,  0.2303, -1.1229],
        [-0.1863,  2.2082, -0.6380,  0.4617,  0.2674],
        [ 0.5349,  0.8094,  1.1103, -1.6898, -0.9890]])
True


### 1.1.2 非生成器

- 设置CPU+当前GPU的随机种子

In [4]:
torch.manual_seed(42)

- 设置当前GPU的随机数种子

In [6]:
torch.cuda.manual_seed(42)

- 设置所有GPU的随机种子

In [5]:
torch.cuda.manual_seed_all(42)

## 1.2 生成

### 1.2.1基础随机张量

In [7]:
# 均匀分布 [0, 1)
x = torch.rand(2, 3)  # 2x3矩阵

# 标准正态分布 (均值0, 方差1)
x = torch.randn(2, 3)

# 均匀分布整数 [low, high)
x = torch.randint(low=0, high=10, size=(2, 3))

# 指定范围内的随机浮点数
x = torch.empty(2, 3).uniform_(0, 1)  # [0, 1)

### 1.2.2特殊分布随机数

In [12]:
# 伯努利分布 (0/1随机)
x = torch.bernoulli(torch.tensor([[0.3, 0.7]]))

# 正态分布 (自定义均值和方差)
x = torch.normal(mean=0.0, std=1.0, size=(2, 3))

# 泊松分布
x = torch.poisson(torch.tensor([1.0, 2.0]))

# 创建张量并原地赋值为指数分布
x = torch.empty(2, 3).exponential_(lambd=1.0)

# 伽马分布
x = torch._sample_dirichlet(torch.tensor([1.0, 2.0]))

### 1.2.3随机排列和选择

In [11]:
# 随机排列 (0到n-1)
x = torch.randperm(10)

# 多项分布采样
weights = torch.tensor([0.1, 0.2, 0.7])
x = torch.multinomial(weights, num_samples=5, replacement=True)

# 随机打乱
tensor = torch.arange(10)
shuffled = tensor[torch.randperm(len(tensor))]

### 1.2.4 torch.distributions 模块
`torch.distributions`模块提供完整的概率分布

# 2. net.train()作用

- 递归设置训练标志  
net.train() 会将自身及所有子模块（即 Sequential 容器内的每一层）的training 属性设为 True。这意味着整个网络都进入训练状态。

- 影响特定层的前向传播行为  
某些层在训练和评估模式下行为不同：
    - Dropout：在训练模式下会随机丢弃神经元（防止过拟合），而在评估模式下会保留所有神经元并缩放权重，以保证输出确定性。
    - BatchNorm：在训练模式下使用当前 mini‑batch 的均值和方差进行归一化，并更新全局 running 统计量；在评估模式下则使用固定的 running 统计量。
    - 其他自定义层如果依赖 self.training 也可能改变行为。

# 3. net.paramaters()

还有一个`named_parameters()`

返回一个生成器,包含所有参数.  
有的层没有paramaters,是空的  
生成器中可以用`next()`获取,每一个是`torch.nn.parameter.Parameter`对象. 不过接口很统一,和torch.Tensor相似